Query GreatSchools for parent ratings - note that a personal API key is requred to work with the GreatSchools API

In [ ]:
import csv 
import requests 
import xml.etree.ElementTree as ET 
import xmltodict
import pandas as pd
import json
import requests
import datetime

In [ ]:
master_list= pd.read_excel('Data/SchoolDirectory/pubschls.xls', sheet_name='schools', header=0)

In [ ]:
charter_list = master_list.loc[master_list['Charter'] == 'Y']
charter_list = charter_list[['CDSCode','Latitude', 'Longitude']]
charter_list = charter_list.reset_index(drop=True)

In [ ]:
number_of_schools = len(charter_list)
print(number_of_schools)

In [ ]:
city = master_list['City']
city.describe()
city = city.drop_duplicates()
city = city.reset_index(drop=True)
city = city.dropna()
city = pd.DataFrame(city)
city = city["City"].str.split(" ", n = 2, expand = True) 
city = city.rename(columns={0: "First", 1: "Second", 2: "Third"})
city.loc[city['Second'].notnull() & city['Third'].notnull() ,'final'] = city['First'] +'+'+ city['Second'] +'+'+ city['Third']
city.loc[city['Second'].notnull() & city['Third'].isna() ,'final'] = city['First'] +'+'+ city['Second']
city.loc[city['Second'].isna() & city['Third'].isna() ,'final'] = city['First']
city_len = len(city)
print(city_len)

In [ ]:
schools_of_interest = []

Because a personal API key is required for querying information using the GreatSchools API - this part of the code cannot be run, proceed below to continue with the construction of the data set.

In [ ]:
for i in range(0, city_len):
    try: 
        school_city = city['final'].loc[i]
        print(school_city)
        # This involves having a personal GreatSchools API key, which is removed here
        #url = "https://api.greatschools.org/schools/nearby?key=####&city={}&state=CA&schoolType=public".format(school_city)

        response = requests.get(url)

        doc = xmltodict.parse(response.text)
        schools = doc['schools']
        school = schools['school']

        for item in school:
            schools_of_interest.append(item['gsId'])
        
    except:
        pass

schools_of_interest = pd.DataFrame(schools_of_interest) 
schools_of_interest = schools_of_interest.drop_duplicates()
schools_of_interest = schools_of_interest.rename(columns={0: "gsId"})

schools_of_interest.to_pickle("Data/GreatSchools/Schools_of_Interest2.pkl")

schools_of_interest = pd.read_pickle("Data/GreatSchools/Schools_of_Interest2.pkl")

school_rating_set = pd.DataFrame(columns=list('ABCDEF'))

for i in range(7500, 8608):
    try: 
        schoolid = schools_of_interest[0][i]
        # This involves having a personal GreatSchools API key, which is removed here
        #url = "https://api.greatschools.org/schools/CA/{}?key=####".format(schoolid)
    
        response = requests.get(url)
    
        doc = xmltodict.parse(response.text)
        school = doc['school']

        school_rating_set.loc[i] = [school['gsRating'],school['parentRating'],school['enrollment'], school['ncesId'], school['gsId'], school['name']]
    
    except:
        pass

school_rating_set.to_pickle("Data/GreatSchools/Schools_Set_Out_7500_8608.pkl")

Begin running code again here

In [ ]:
school_rating_1 = pd.read_pickle("Data/GreatSchools/Schools_Set_Out_0_499.pkl")
school_rating_2 = pd.read_pickle("Data/GreatSchools/Schools_Set_Out_500_2900.pkl")
school_rating_3 = pd.read_pickle("Data/GreatSchools/Schools_Set_Out_2900_4800.pkl")
school_rating_4 = pd.read_pickle("Data/GreatSchools/Schools_Set_Out_4800_7500.pkl")
school_rating_5 = pd.read_pickle("Data/GreatSchools/Schools_Set_Out_7500_8608.pkl")

In [ ]:
school_rating_a = school_rating_1.append(school_rating_2)
school_rating_b = school_rating_a.append(school_rating_3)
school_rating_c = school_rating_b.append(school_rating_4)
school_rating_d = school_rating_c.append(school_rating_5)


In [ ]:
school_rating = school_rating_d.reset_index(drop=True)
school_rating = school_rating.rename(columns={"A": "gs_Rating", "B": "parentRating", "C": "enrollment", "D": "ncesId", "E": "gsId", "F" : "name"})

In [ ]:
Full_Set = pd.read_pickle("Data/SchoolDirectory/All_Schools_Analysis.pkl")

In [ ]:
Full_Set['ncesId'] = Full_Set['NCESDist'] + Full_Set['NCESSchool']
Charter_Schools = Full_Set.loc[Full_Set['Charter'] == "Y"]
Charter_Schools2 = Charter_Schools[['School', 'CDS_CODE', 'ncesId', 'Latitude', 'Longitude']]
Charter_Schools3= Charter_Schools2.drop_duplicates(subset='CDS_CODE')

In [ ]:
Public_Schools = Full_Set.merge(school_rating, left_on='ncesId', right_on='ncesId')
Public_Schools2 = Public_Schools.drop_duplicates(subset='CDS_CODE')
Public_Schools3 = Public_Schools2[['School', 'CDS_CODE', 'ncesId', 'Latitude', 'Longitude', 'enrollment', 'gs_Rating', 'parentRating']]

In [ ]:
Public_Schools_Final = Public_Schools3.reset_index(drop=True)
Charter_Schools_Final = Charter_Schools3.reset_index(drop=True)

Public_Schools_Final.to_pickle("Data/GreatSchools/Public_Schools.pkl")
Charter_Schools_Final.to_pickle("Data/GreatSchools/Charter_Schools.pkl")

In [ ]:
import geopy.distance as geodesic
import pickle
import random
import os

import pandas

In [ ]:
lat_delta_mile = 0.02
lon_delta_mile = 0.025

#Get Schools
charter_schools = pickle.load(open("Data/GreatSchools/Charter_Schools.pkl", 'rb'))
public_schools = pickle.load(open("Data/GreatSchools/Public_Schools.pkl", 'rb'))

#Create school weights
public_schools['gs_weight'] = pandas.to_numeric(public_schools.gs_Rating) * pandas.to_numeric(public_schools.enrollment)
public_schools['ps_weight'] = pandas.to_numeric(public_schools.parentRating) * pandas.to_numeric(public_schools.enrollment)

charter_schools_dict = pandas.DataFrame.to_dict(charter_schools)
public_schools_dict = pandas.DataFrame.to_dict(public_schools)

#def schools_distance(charter_schools_dict, public_schools_dict, range = 3):
range = 3
charter_schools_dict["public_schools_in_range_gs_weight"] = {}
charter_schools_dict["public_schools_in_range_ps_weight"] = {}
charter_schools_dict["public_schools_in_range_total"] = {}
charter_schools_dict["neighborhood_GS_score"] = {}
charter_schools_dict["neighborhood_PS_score"] = {}

for charter in charter_schools_dict["School"]:
	print(charter)
	charter_schools_dict["public_schools_in_range_gs_weight"][charter] = 0
	charter_schools_dict["public_schools_in_range_ps_weight"][charter] = 0
	charter_schools_dict["public_schools_in_range_total"][charter] = 0
	c_loc = (charter_schools_dict["Latitude"][charter], charter_schools_dict["Longitude"][charter])

	for public in public_schools_dict["School"]:
		p_loc = (public_schools_dict["Latitude"][public], public_schools_dict["Longitude"][public])

		if not ('No Data' in c_loc or "No Data" in p_loc):

#			test_lat_delta = abs(float(c_loc[0]) - float(p_loc[0]))
#			test_lon_delta = abs(float(c_loc[1]) - float(p_loc[1]))

			#Do a dummy test on distance
#			if not (lat_delta_mile > (test_lat_delta/range)) and not (lon_delta_mile > (test_lon_delta/range)):

			distance = geodesic.geodesic(p_loc, c_loc).mi
			if distance <= range:
				charter_schools_dict["public_schools_in_range_gs_weight"][charter] += public_schools_dict["gs_weight"][public]
				charter_schools_dict["public_schools_in_range_ps_weight"][charter] += public_schools_dict["ps_weight"][public]
				charter_schools_dict["public_schools_in_range_total"][charter] += float(public_schools_dict["enrollment"][public])
				#print(charter_schools_dict["public_schools_in_range_total"][charter])

	if charter_schools_dict["public_schools_in_range_total"][charter] > 0:
		charter_schools_dict["neighborhood_GS_score"][charter] = charter_schools_dict["public_schools_in_range_gs_weight"][charter]/charter_schools_dict["public_schools_in_range_total"][charter]
		charter_schools_dict["neighborhood_PS_score"][charter] = charter_schools_dict["public_schools_in_range_ps_weight"][charter]/charter_schools_dict["public_schools_in_range_total"][charter]
	else:
		charter_schools_dict["neighborhood_GS_score"][charter] = 'NA'
		charter_schools_dict["neighborhood_PS_score"][charter] = 'NA'

School_Neighbors = pandas.DataFrame.from_dict(charter_schools_dict)

pickle.dump(School_Neighbors, open( "School_Neighbors.pkl", "wb" ) )